In [ ]:
#!/usr/bin/env python3
"""
Validación TFM - Script para validar algoritmo UMAP+GMM con casos reales.

Valida si el algoritmo recomienda las sustituciones que realmente se ejecutaron.
Para cada caso, entrena UMAP+GMM con pool de temporada específica y verifica si
el jugador fichado aparece en top N similares al jugador vendido.
"""

import sys
import os
import numpy as np
sys.path.append('/home/jaime/FD/data')

import pandas as pd
from database.connection import get_db_manager
from similarity import DataPreparator, FeatureEngineer, UMAPReducer, GMMClusterer, PlayerSimilarity
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Casos TFM con VENDIDO y FICHADO - IDs verificados en BD
TFM_CASES = [
    # Villarreal - Delanteros
    {
        "vendido": "Nicolas Jackson",
        "vendido_team": "Villarreal",
        "vendido_id": "df3c607f8f46ffc1",
        "fichado": "Alexander Sørloth",
        "fichado_team": "Real Sociedad",
        "fichado_id": "cad28b020b20f985",
        "season": "2223",
        "position": "FW",
        "descripcion": "Jackson → Sørloth"
    },
    {
        "vendido": "Ben Brereton",
        "vendido_team": "Villarreal",
        "vendido_id": "859f713390732d2a",
        "fichado": "Ayoze Pérez",
        "fichado_team": "Betis",
        "fichado_id": "1dc1d9419b82ed73",
        "season": "2324",
        "position": "FW",
        "descripcion": "Brereton → Ayoze"
    },
    {
        "vendido": "Yeremi Pino",
        "vendido_team": "Villarreal",
        "vendido_id": "f096ea6851899e57",
        "fichado": "Georges Mikautadze",
        "fichado_team": "Lyon",
        "fichado_id": "a8fe4e1b129cf192",
        "season": "2425",
        "position": "FW",
        "descripcion": "Pino → Mikautadze"
    },
    # Villarreal - Defensas
    {
        "vendido": "Pervis Estupiñán",
        "vendido_team": "Villarreal",
        "vendido_id": "cbb3780ec36be9be",
        "fichado": "Johan Mojica",
        "fichado_team": "Elche",
        "fichado_id": "c230785304a6bacb",
        "season": "2122",
        "position": "DF",
        "descripcion": "Estupiñán → Mojica"
    },
    {
        "vendido": "Pau Torres",
        "vendido_team": "Villarreal",
        "vendido_id": "7038310ad2ae51db",
        "fichado": "Logan Costa",
        "fichado_team": "Toulouse",
        "fichado_id": "3b445a031f685465",
        "season": "2324",
        "position": "DF",
        "descripcion": "Pau Torres → Costa"
    },
    {
        "vendido": "Johan Mojica",
        "vendido_team": "Elche",
        "vendido_id": "c230785304a6bacb",
        "fichado": "Sergi Cardona",
        "fichado_team": "Las Palmas",
        "fichado_id": "dc5dd1f512cb8bf2",
        "season": "2324",
        "position": "DF",
        "descripcion": "Mojica → Cardona"
    },
    # Villarreal - Mediocampistas
    {
        "vendido": "Álex Baena",
        "vendido_team": "Villarreal",
        "vendido_id": "eb4c447d1a00eb39",
        "fichado": "Alberto Moleiro",
        "fichado_team": "Las Palmas",
        "fichado_id": "e3039aff904c9c54",
        "season": "2425",
        "position": "MF",
        "descripcion": "Baena → Moleiro"
    },
    # Frankfurt
    {
        "vendido": "Randal Kolo Muani",
        "vendido_team": "Frankfurt",
        "vendido_id": "8cfbfca3e5dc4ba0",
        "fichado": "Omar Marmoush",
        "fichado_team": "Wolfsburg",
        "fichado_id": "b5fd95dbeba1f61f",
        "season": "2223",
        "position": "FW",
        "descripcion": "Kolo Muani → Marmoush"
    },
    {
        "vendido": "Randal Kolo Muani",
        "vendido_team": "Frankfurt",
        "vendido_id": "8cfbfca3e5dc4ba0",
        "fichado": "Hugo Ekitike",
        "fichado_team": "Paris S-G",
        "fichado_id": "2af83c0acb2812de",
        "season": "2223",
        "position": "FW",
        "descripcion": "Kolo Muani → Ekitike (cedido)"
    },
    {
        "vendido": "Omar Marmoush",
        "vendido_team": "Frankfurt",
        "vendido_id": "b5fd95dbeba1f61f",
        "fichado": "Elye Wahi",
        "fichado_team": "Lens",
        "fichado_id": "588046b2ff0315c7",
        "season": "2324",
        "position": "FW",
        "descripcion": "Marmoush → Wahi"
    },
    {
        "vendido": "Hugo Ekitike",
        "vendido_team": "Frankfurt",
        "vendido_id": "2af83c0acb2812de",
        "fichado": "Jonathan Burkardt",
        "fichado_team": "Mainz 05",
        "fichado_id": "b6fcc5ac2ef92f29",
        "season": "2425",
        "position": "FW",
        "descripcion": "Ekitike → Burkardt"
    },
    {
        "vendido": "Willian Pacho",
        "vendido_team": "Frankfurt",
        "vendido_id": "3b6c6d66fee0938d",
        "fichado": "Arthur Theate",
        "fichado_team": "Rennes",
        "fichado_id": "4736a05f4cc311c0",
        "season": "2324",
        "position": "DF",
        "descripcion": "Pacho → Theate"
    },
    # Lille - Caso fallido
    {
        "vendido": "Carlos Baleba",
        "vendido_team": "Lille",
        "vendido_id": "f78eb44040a1bfde",
        "fichado": "Nabil Bentaleb",
        "fichado_team": "Angers",
        "fichado_id": "ee0c963766e4e66c",
        "season": "2223",
        "position": "MF",
        "descripcion": "Baleba → Bentaleb (caso fallido)"
    },
]


def validate_single_case(db, case: dict) -> dict:
    """
    Valida un caso individual del TFM siguiendo EXACTAMENTE el flujo del template.
    
    Proceso:
    1. Cargar pool temporada actual (5 ligas) - SOLO UNA TEMPORADA
    2. Extraer métricas siguiendo template
    3. Feature engineering siguiendo template
    4. UMAP + GMM siguiendo template
    5. Buscar similares y validar
    
    Returns:
        dict con resultados validación
    """
    logger.info(f"\n{'='*80}")
    logger.info(f"VALIDANDO: {case['descripcion']}")
    logger.info(f"Temporada analizada: {case['season']}")
    logger.info(f"{'='*80}\n")
    
    try:
        # 1. Usar SOLO la temporada actual (sin temp-1)
        season_current = case['season']
        
        logger.info(f"Pool temporada: {season_current}")
        
        # 2. Cargar datos pool (5 ligas) - SOLO UNA TEMPORADA
        leagues = ['ESP-La Liga', 'ENG-Premier League', 'GER-Bundesliga',
                   'FRA-Ligue 1', 'ITA-Serie A']
        position = case['position']
        
        # Cargar cada liga y concatenar MANUALMENTE
        df_list = []
        for league in leagues:
            try:
                data_prep_temp = DataPreparator(db, table_type='domestic')
                df_temp = data_prep_temp.load_players(
                    leagues=[league],
                    season=season_current,
                    position_filter=position,
                    min_minutes=400
                )
                if not df_temp.empty:
                    df_list.append(df_temp)
                    logger.debug(f"  Cargado: {league} {season_current} - {len(df_temp)} jugadores")
            except Exception as e:
                logger.debug(f"  No data: {league} {season_current} - {e}")
                continue
        
        if not df_list:
            logger.error(f"✗ No data for {position} in season {season_current}")
            return {'status': 'NO_DATA', **case}
        
        # Concatenar ANTES de crear el DataPreparator final
        df_raw_concat = pd.concat(df_list, ignore_index=True)
        logger.info(f"Pool cargado: {len(df_raw_concat)} jugadores (pre-limpieza)")

        # Crear DataPreparator y establecer df_raw manualmente
        data_prep = DataPreparator(db, table_type='domestic')
        data_prep.set_raw_data(df_raw_concat)

        # 3. EXTRACCION METRICAS - siguiendo template
        df_metrics = data_prep.extract_all_metrics()
        logger.info(f"Métricas extraídas: {df_metrics.shape[1]} columnas, {len(df_metrics)} jugadores")

        # 4. MANEJO VALORES FALTANTES - siguiendo template
        df_clean = data_prep.handle_missing_values(
            strategy='median_by_position',
            max_missing_pct=0.4
        )
        logger.info(f"Datos limpios: {df_clean.shape}")

        # 5. DETECCION OUTLIERS - siguiendo template
        df_outliers = data_prep.detect_outliers(
            method='isolation_forest',
            contamination=0.05
        )
        logger.info(f"Outliers detectados: {df_outliers['is_outlier'].sum()}")

        # 6. FEATURE ENGINEERING - siguiendo template
        feature_eng = FeatureEngineer(position_type=position)
        
        df_selected = feature_eng.select_relevant_features(
            df_outliers,
            exclude_gk_metrics=True,
            min_variance=0.01
        )
        logger.info(f"Features seleccionadas: {len(feature_eng.selected_features)}")

        # 7. ELIMINAR FEATURES CORRELACIONADAS - siguiendo template
        df_uncorrelated = feature_eng.remove_correlated_features(
            df_selected,
            threshold=0.95
        )
        logger.info(f"Features no redundantes: {len(feature_eng.selected_features)}")

        # 8. NORMALIZACION POR POSICION - siguiendo template
        df_normalized = feature_eng.normalize_by_position(
            df_uncorrelated,
            method='standard',
            fit_per_position=True
        )
        logger.info("Normalización completada")

        # 9. PREPARAR MATRIZ FEATURES PARA UMAP - siguiendo template
        X, metadata_df = feature_eng.prepare_for_umap(df_normalized, return_dataframe=True)
        logger.info(f"Matriz features: {X.shape}")

        # 10. REDUCCION DIMENSIONAL UMAP - siguiendo template
        umap_reducer = UMAPReducer(
            n_components=5,
            n_neighbors=20,
            min_dist=0.0,
            metric='euclidean',
            random_state=42
        )
        X_umap = umap_reducer.fit_transform(X, verbose=False)
        logger.info(f"UMAP embedding: {X_umap.shape}")

        # 11. CREAR DATAFRAME EMBEDDING - siguiendo template
        embedding_df = umap_reducer.get_embedding_dataframe(metadata_df)
        
        # 12. CLUSTERING GMM - ENCONTRAR N OPTIMO - siguiendo template
        gmm_clusterer = GMMClusterer(
            covariance_type='full',
            max_iter=200,
            random_state=42
        )
        optimal_results = gmm_clusterer.find_optimal_clusters(
            X_umap,
            min_clusters=3,
            max_clusters=12,
            criterion='bic'
        )
        logger.info(f"Número óptimo clusters: {optimal_results['optimal_n']}")

        # 13. FIT GMM CON N OPTIMO - siguiendo template
        gmm_clusterer.fit(X_umap, n_components=optimal_results['optimal_n'])
        logger.info(f"GMM fitted con {gmm_clusterer.n_components} clusters")

        # 14. INICIALIZAR MOTOR SIMILITUD - siguiendo template
        similarity_engine = PlayerSimilarity(
            embedding_df=embedding_df,
            gmm_proba=gmm_clusterer.labels_proba,
            feature_df=df_normalized,
            weights={'umap_distance': 0.50, 'gmm_probability': 0.30, 'feature_similarity': 0.20}
        )
        logger.info("Motor similitud inicializado")

        # 15. BUSCAR VENDIDO EN POOL
        vendido_id = case.get('vendido_id')
        
        if vendido_id not in embedding_df['unique_player_id'].values:
            logger.warning(f"✗ Vendido ID {vendido_id} no en pool tras limpieza")
            return {'status': 'VENDIDO_NOT_IN_EMBEDDING', **case}
        
        logger.info(f"✓ Vendido encontrado: {case['vendido']} (ID: {vendido_id})")

        # 16. BUSCAR SIMILARES - siguiendo template
        similar_players = similarity_engine.find_similar_players(
            player_identifier=vendido_id,
            top_n=50,
            filters={'exclude_same_team': False},
            return_scores=True
        )
        
        if similar_players.empty:
            logger.warning(f"✗ No similar players found")
            return {'status': 'NO_SIMILAR', **case}

        # 17. VERIFICAR SI FICHADO ESTÁ EN TOP N
        fichado_id = case.get('fichado_id')
        
        fichado_rank = None
        fichado_score = None
        
        for idx, row in similar_players.iterrows():
            if row['unique_player_id'] == fichado_id:
                fichado_rank = idx + 1
                fichado_score = row['similarity_score']
                break

        # Mostrar top 10
        logger.info(f"\nTop 10 similares a {case['vendido']}:")
        for idx, row in similar_players.head(10).iterrows():
            marca = "★" if row['unique_player_id'] == fichado_id else " "
            logger.info(f"  {marca} {idx+1:2}. {row['player_name']:25} ({row['team']:20}) Score: {row['similarity_score']:.3f}")

        if fichado_rank:
            logger.info(f"\n✓ FICHADO ENCONTRADO: {case['fichado']} en posición #{fichado_rank} (score: {fichado_score:.3f})")
            return {
                'status': 'SUCCESS',
                'fichado_rank': fichado_rank,
                'fichado_score': fichado_score,
                'top_10': similar_players.head(10)[['player_name', 'team', 'similarity_score']].to_dict('records'),
                **case
            }
        else:
            logger.warning(f"\n⚠ Fichado '{case['fichado']}' NO en top 50")
            return {
                'status': 'FICHADO_NOT_IN_TOP',
                'top_10': similar_players.head(10)[['player_name', 'team', 'similarity_score']].to_dict('records'),
                **case
            }
    
    except Exception as e:
        logger.error(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return {'status': 'ERROR', 'error': str(e), **case}


def main():
    """Ejecutar validación TFM."""
    logger.info("="*80)
    logger.info("VALIDACIÓN TFM - ALGORITMO UMAP+GMM")
    logger.info("Validando sustituciones reales ejecutadas por equipos")
    logger.info("="*80 + "\n")
    
    db = get_db_manager()
    logger.info("✓ Conectado a BD\n")
    
    results = []
    
    for i, case in enumerate(TFM_CASES, 1):
        logger.info(f"\n[{i}/{len(TFM_CASES)}] {case['descripcion']}")
        result = validate_single_case(db, case)
        results.append(result)
    
    # Métricas finales
    logger.info("\n" + "="*80)
    logger.info("RESUMEN FINAL")
    logger.info("="*80 + "\n")
    
    success = [r for r in results if r['status'] == 'SUCCESS']
    fichado_not_top = [r for r in results if r['status'] == 'FICHADO_NOT_IN_TOP']
    fichado_not_found = [r for r in results if r['status'] == 'FICHADO_NOT_FOUND']
    vendido_not_found = [r for r in results if r['status'] == 'VENDIDO_NOT_FOUND']
    vendido_not_embedding = [r for r in results if r['status'] == 'VENDIDO_NOT_IN_EMBEDDING']
    errors = [r for r in results if r['status'] == 'ERROR']
    
    logger.info(f"✓ Casos exitosos (fichado en top 50): {len(success)}/{len(TFM_CASES)}")
    logger.info(f"⚠ Fichado no en top 50: {len(fichado_not_top)}")
    logger.info(f"✗ Fichado no encontrado en pool: {len(fichado_not_found)}")
    logger.info(f"✗ Vendido no encontrado en pool: {len(vendido_not_found)}")
    logger.info(f"✗ Vendido eliminado en limpieza: {len(vendido_not_embedding)}")
    logger.info(f"✗ Errores: {len(errors)}")
    
    if success:
        logger.info(f"\nCasos exitosos:")
        for r in success:
            logger.info(f"  ✓ {r['descripcion']:30} → Rank #{r['fichado_rank']:2} (score: {r['fichado_score']:.3f})")
    
    if fichado_not_top:
        logger.info(f"\nFichado no en top 50:")
        for r in fichado_not_top:
            logger.info(f"  ⚠ {r['descripcion']}")
    
    db.close()
    logger.info("\n✓ Validación completada\n")
    
    return results


if __name__ == "__main__":
    results = main()